In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")
sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(11.7, 8.27)})

In [3]:
pwd()

'/Users/g9385/code/music-streaming/notebooks'

In [4]:
e = create_engine('sqlite:///../data/processed/music.db', echo=False)

In [7]:
sql_top_users = """
SELECT
    ROW_NUMBER() OVER () as rnk
    , user_name
    , n_listens
FROM (
    SELECT
        du.user_name
        , sum(ku.listens) as n_listens
    FROM kpi_user AS ku
    INNER JOIN dim_user AS du ON (ku.user_id = du.id)
    GROUP BY
        ku.user_id
    ORDER BY
        n_listens DESC
    LIMIT 10
);
"""

In [8]:
df_top_users = pd.read_sql_query(sql_top_users, e)
df_top_users.head()

,rnk,user_name,n_listens
0,1,hds,46880
1,2,Groschi,14959
2,3,Silent Singer,12888
3,4,6d6f7274686f6e,11544
4,5,phdnk,11533


In [9]:
sql_active_count = """
SELECT
    users
FROM kpi_listen
WHERE listened_at = DATE('2019-03-01');
"""

In [10]:
df_active_count = pd.read_sql_query(sql_active_count, e)
df_active_count.head()

,users
0,75


In [11]:
sql_first_song = """
WITH users AS (
    SELECT
        user_id
        , min(listened_at) AS listened_at
    FROM fact_listen
    GROUP BY
        user_id
)
SELECT
    fl.listened_at
    , fl.user_id
    , dt.track_name
FROM fact_listen AS fl
LEFT JOIN users AS u ON (fl.user_id = u.user_id AND fl.listened_at = u.listened_at)
LEFT JOIN dim_track AS dt ON (fl.track_id = dt.id)
ORDER BY
    fl.listened_at ASC;
"""

In [12]:
df_first_song = pd.read_sql_query(sql_first_song, e)
df_first_song.head()

,listened_at,user_id,track_name
0,2019-01-01 00:00:48.000000,75,Shitstorm
1,2019-01-01 00:02:42.000000,32,Crystallized
2,2019-01-01 00:03:08.000000,51,Disco Yes
3,2019-01-01 00:03:08.000000,77,Про красивую жизнь
4,2019-01-01 00:04:37.000000,162,Never Gonna Make (Factory Dance Team mix)
